# Exploratory Data Analysis of Dataset

## Step 1: Data Profiling (EDA)

1. Check for missing values.

2. Identify data types.

3. Detect duplicates.

4. Perform summary statistics for numerical & categorical columns.

5. Analyze distribution of ratings and customer feedback.

In [2]:
import nltk

# Download necessary NLTK datasets
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\SachithN\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\SachithN\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\SachithN\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [6]:
import pandas as pd
df = pd.read_csv("EDA.csv")

In [ ]:
%pip install missingno
import missingno as msno

# Convert date columns to datetime
df['CreatedOn'] = pd.to_datetime(df['CreatedOn'], errors='coerce')

# Display missing values heatmap
msno.matrix(df)

plt.show()

In [9]:
df.head()

,FeedbackCode,FeedbackId,DivisionName,InvoiceNo,CustomerId,CustomerName,ContactNo,Location,Designation,JobNumber,...,IsRecommended,Review,ReviewAny,CreatedOn,CreatedBy,DeletedBy,DeletedOn,UpdatedBy,UpdatedOn,IsActive


In [10]:
# Identify missing values
missing_values = df.isnull().sum()


In [11]:
# Drop irrelevant columns
drop_columns = ['DeletedBy', 'DeletedOn', 'UpdatedBy', 'UpdatedOn', 'IsActive']
df = df.drop(columns=drop_columns, errors='ignore')

In [12]:


# Handle missing values
df['CustomerName'].fillna('Unknown', inplace=True)
df['Location'].fillna('Unknown', inplace=True)
df['ReviewAny'].fillna('', inplace=True)



C:\Users\SachithN\AppData\Local\Temp\ipykernel_11596\888139176.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['CustomerName'].fillna('Unknown', inplace=True)
C:\Users\SachithN\AppData\Local\Temp\ipykernel_11596\888139176.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

F

In [13]:
# Replace missing numerical values with median
num_cols = ['Rate', 'RateofSatisfyService', 'RateofTimeTaken', 'RateofServiceProduct']
for col in num_cols:
    if col in df.columns:
        df[col] = df[col].fillna(df[col].median())

In [ ]:
import nltk
nltk.data.path.append('/custom/path/to/nltk_data')
nltk.download('punkt', download_dir='/custom/path/to/nltk_data')

In [ ]:

# Ensure 'Cleaned_Review' exists; if not, create it using 'ReviewAny'
if 'Cleaned_Review' not in df.columns:
	df['Cleaned_Review'] = df['ReviewAny']

# Generate a Word Cloud for Reviews
wordcloud = WordCloud(width=800, height=400, background_color="white").generate(" ".join(df['Cleaned_Review']))
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.title("Word Cloud of Customer Reviews")
plt.show()

# Sentiment Analysis using TextBlob
df['Sentiment'] = df['Cleaned_Review'].apply(lambda x: TextBlob(x).sentiment.polarity)
df['Sentiment_Label'] = df['Sentiment'].apply(lambda x: 'Positive' if x > 0 else ('Negative' if x < 0 else 'Neutral'))

# Display correlation heatmap (using actual rating columns)
rating_cols = [
	'How satisfied are you with the service/product quality?', 
	'How satisfied are you with the behavior of the service / sales person?', 
	'How likely would you recommend Browns products & services to others?', 
	'How would you rate the overall experience of the service provided by Browns?'
]
plt.figure(figsize=(8, 5))
sns.heatmap(df[rating_cols].corr(), annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Correlation Heatmap of Ratings")
plt.show()




In [ ]:
df.to_csv("preprocessed_feedback_data.csv", index=False)
print("Data saved as preprocessed_feedback_data.csv")


In [ ]:
df.head()

## Step 2: Data Cleaning & Preprocessing
1. Handle missing values:
For numerical columns → fill with mean/median.
For categorical columns → fill with mode or "Unknown".
For text (review comments) → drop if empty or use placeholder.

2. Convert date columns (CreatedOn) to datetime format.

3. Drop irrelevant columns like DeletedBy, DeletedOn (if all null or not needed).

4. Standardize text data in ReviewAny (lowercasing, punctuation removal, stopwords removal).

5. Convert categorical data to numeric encoding (if needed for ML models).

6. Check for outliers in ratings.

In [15]:
df = pd.read_csv("preprocessed_feedback_data.csv")

In [ ]:
df.head()

In [ ]:
# Compute the percentage of null values for each column in the DataFrame
null_percentage = df.isnull().mean() * 100
print(null_percentage)

In [ ]:
# 1. Handle missing values

# For numerical rating columns, fill missing values with the median.
for rating in rating_cols:
    if rating in df.columns:
        df[rating].fillna(df[rating].median(), inplace=True)

# For categorical columns (except for review texts and dates), fill missing values with the mode.
categorical_cols = df.select_dtypes(include='object').columns
for col_cat in categorical_cols:
    if col_cat not in ['ReviewAny', 'Cleaned_Review', 'CreatedOn']:
        if df[col_cat].isnull().sum() > 0:
            mode_val = df[col_cat].mode()[0] if not df[col_cat].mode().empty else "Unknown"
            df[col_cat].fillna(mode_val, inplace=True)

# For text review columns, fill empty or missing entries with a placeholder.
if 'ReviewAny' in df.columns:
    df['ReviewAny'] = df['ReviewAny'].apply(lambda x: x if isinstance(x, str) and x.strip() != "" else "No review provided")


In [19]:

# 2. Convert 'CreatedOn' to datetime format.
if 'CreatedOn' in df.columns:
    df['CreatedOn'] = pd.to_datetime(df['CreatedOn'], errors='coerce')


In [20]:

# 3. Drop irrelevant columns if they exist.
irrelevant = ['DeletedBy', 'DeletedOn', 'UpdatedBy', 'UpdatedOn', 'IsActive']
df.drop(columns=[col for col in irrelevant if col in df.columns], inplace=True)


In [21]:

# 5. Convert categorical columns to numeric encoding where appropriate.
# We'll factorize object columns (excluding date and review texts) that have a relatively small number of unique values.
for col_cat in df.select_dtypes(include='object').columns:
    if col_cat not in ['CreatedOn', 'ReviewAny', 'Cleaned_Review']:
        if df[col_cat].nunique() < 100:  # arbitrary threshold for categorical encoding
            df[col_cat], _ = pd.factorize(df[col_cat])


In [ ]:

# 6. Check for outliers in the ratings using boxplots.
for rating in rating_cols:
    if rating in df.columns:
        plt.figure()
        sns.boxplot(x=df[rating])
        plt.title(f"Boxplot for {rating}")
        plt.show()

## Step 3: Sentiment Analysis Preprocessing
1. Use NLP techniques to prepare text for analysis:
2. Tokenization
3. Stopword removal
4. Lemmatization
5. Apply sentiment scoring (VADER, TextBlob) for text reviews

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')


In [ ]:
%pip install vaderSentiment

import pandas as pd
import re
import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Load the preprocessed feedback data
import pandas as pd
import re
import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from textblob import TextBlob



In [ ]:

# Ensure necessary NLTK resources are available
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Initialize NLP tools
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

# Function to preprocess text
def preprocess_text(text):
    text = str(text).lower()  # Convert to lowercase
    text = re.sub(r'\d+', '', text)  # Remove numbers
    text = text.translate(str.maketrans("", "", string.punctuation))  # Remove punctuation
    tokens = word_tokenize(text)  # Tokenization
    tokens = [word for word in tokens if word not in stop_words]  # Stopword removal
    tokens = [lemmatizer.lemmatize(word) for word in tokens]  # Lemmatization
    return " ".join(tokens)


In [ ]:
import nltk
nltk.download('punkt_tab')
df = pd.read_csv("preprocessed_feedback_data.csv")
# Apply preprocessing
df['Cleaned_Review'] = df['ReviewAny'].apply(preprocess_text)

# Apply sentiment scoring (TextBlob only)
df['TextBlob_Sentiment'] = df['Cleaned_Review'].apply(lambda x: TextBlob(x).sentiment.polarity)

# Categorize sentiment
df['Sentiment_Label'] = df['TextBlob_Sentiment'].apply(lambda x: 'Positive' if x > 0 else ('Negative' if x < 0 else 'Neutral'))

df.head()

In [27]:
df.to_csv('data4final.csv', index=False)
print("Final CSV saved as data4final.csv")

Final CSV saved as data4final.csv
